### ** 포털공사관리번호 필요 ptlCmno (설계VE 목록 조회)

### 1. 기본 수집정보 불러오기

In [1]:
### 기본라이브러리 정의
from common import commonFunc as cf
import pandas as pd
import pickle

metadata = pd.read_excel("../input/datalake_meta22.xlsx")

### 기본정보 설정 및 파라미터 설정 ###
SITENAME = "건설사업정보시스템"
DATANAME= "설계VE 계약정보"
# 기존: APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

In [2]:
APIKEYLEN = len(APIKEY)

In [3]:
APIKEYLEN

3

In [4]:
targetData = metadata.loc[metadata.자료명==DATANAME]
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
### 기본정보 설정 및 파라미터 설정 ###

### 2. 입력 파라미터 정보 불러오기

In [6]:
### 기본정보 설정 및 파라미터 설정 ###
PREDATANAME= "설계VE 목록 조회"
preData = metadata.loc[metadata.자료명==PREDATANAME]

preDt = preData.저장폴더.values[0]
preDtResult = pd.read_csv(preDt, encoding="ms949")
newParam = preDtResult.ptlcmno.drop_duplicates().tolist()

In [7]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###

### [추가] 3. 파라미터정보 불러오기

In [9]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
    outData = cf.loaddata(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output\\건설사업정보시스템\\설계VE 계약정보\\selectIoPtVeContractCorpList.pickle'
설계VE 계약정보 정보 new 모드 2 


### 데이터 수집

In [10]:
### 개별 코드 작업 영역 ###
### baseparam은 수동설정필요

ptlcmnoListLen = len(ptlcmnoList)
resultDfMerged = pd.DataFrame()
pageList = [] 

# ptlcmnoListLen = len(ptlcmnoList)
APICALL = 0

for i in range(0, ptlcmnoListLen):
    apiNo = i%APIKEYLEN
    BASEPARAM={"serviceKey":APIKEY[0], "type":"json", "ptlCmno": ptlcmnoList[i]} #변경 apiNo -> 0
    # scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN,APIKEY,APICALL) #추가
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]  #추가
    resultDfMerged = resultDfMerged.append(resultDf)

1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe설계VE 계약정보, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'type': 'json', 'ptlCmno': 'B5000272014043'} rows: 9 completed
1 page scraping start apicall iter: 2 / used 9D486CED-81E4-4884-96FF-BB1F14A9223B
1 page is empty
dataframe설계VE 계약정보, param:{'serviceKey': '9D486CED-81E4-4884-96FF-BB1F14A9223B', 'type': 'json', 'ptlCmno': 'B6000002021007'} rows: 0 completed
1 page scraping start apicall iter: 3 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe설계VE 계약정보, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'type': 'json', 'ptlCmno': 'B6000002017057'} rows: 9 completed
1 page scraping start apicall iter: 4 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe설계VE 계약정보, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'type': 'json', 'ptlCmno': 'B6000002019057'} rows: 9 completed
1 page scraping start apicall iter: 5 / used

### [추가] 파라미터 정보 추가저장

In [11]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME, mode)

resultDfMerged.shape

설계VE 계약정보 save compled


(3167, 9)

In [12]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)

In [ ]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)